In [ ]:
# Training data has a size of N/8

In [ ]:
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, Lambda
from keras.layers import Conv2DTranspose, BatchNormalization, Activation, Dropout, UpSampling2D, Conv2D
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import tensorflow as tf
import numpy as np
from numpy import asarray
from numpy import expand_dims
from numpy import linalg
from PIL import Image
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image_size = 160
image_reduction_factor = 8

In [ ]:
tf.__version__

In [ ]:
model_path = './model/keras/model/facenet_keras.h5'
model_FN = load_model(model_path)

In [ ]:
# Make the model non-trainable
for l in model_FN.layers:
    l.trainable = False

In [ ]:
def prewhiten(x):

    if K.ndim(x) == 4:
        axis = (1, 2, 3)
        size = K.int_shape(x)
        size = size[1]*size[2]*size[3]
    elif K.ndim(x) == 3:
        axis = (0, 1, 2)
        size = K.int_shape(x)
        size = size[0]*size[1]*size[2]
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = K.mean(x, axis=axis, keepdims=True)
    std = K.std(x, axis=axis, keepdims=True)
    std_adj = K.maximum(std, 1.0/np.sqrt(size)) 
    y = (x - mean) / std_adj
    return y

In [ ]:
# Load Training, Dev, and Test data
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')

X_dev = np.load('X_dev.npy')
Y_dev = np.load('Y_dev.npy')

X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')

In [ ]:
# Check the data sizes
print(X_train.shape)
print(Y_train.shape)
print(X_dev.shape)
print(Y_dev.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
# Start the development here:
# Define the upsampling model
input_layer = Input(shape=(20,20,3), name='model_input_UR')
# Need to turn the input tensor from (20,20,3) to (160,160,3)
x = input_layer
x = Conv2D(filters=256, kernel_size=3, strides=1, padding='same', name='decoder_conv_t_0_UR')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = UpSampling2D()(x) #40
x = Conv2D(filters=128, kernel_size=3, strides=1, padding='same', name='decoder_conv_t_1_UR')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = UpSampling2D()(x) #80
x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', name='decoder_conv_t_2_UR')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = UpSampling2D()(x) #160
x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', name='decoder_conv_t_3_UR')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(filters=3, kernel_size=3, strides=1, padding='same', name='decoder_conv_t_4_UR')(x)
output_layer = Activation('sigmoid')(x)

model_UR = Model(input_layer, output_layer)
model_UR.summary()

In [ ]:
# Define the final model
input_layer_final = Input(shape=(20,20,3), name='model_input_final')
x = input_layer_final
# 1. Pass through the training NN
x = model_UR(x)
# 2. Prewhitening before passing to Facenet 
x = Lambda(lambda x: prewhiten(x))(x)
# 3. Pass through the non-trainable facenet 
x = model_FN(x)
# 4. Pass through the L2-norm layer : K.l2_normalize(X, axis=1)
output_layer_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
# 5. Define the Model
model_final = Model(input_layer_final, output_layer_final)
model_final.summary()

In [ ]:
# 5. Compile the Model
opt = Adam(lr=0.005)
model_final.compile(loss='mse', optimizer=opt, metrics=['mae'])

# 6. Training the Model
tensorboard = TensorBoard('./logs/UpSam')
checkpoint = ModelCheckpoint('model_final_sigmoid_N8_{epoch:03d}.h5', period=5)
model_final.fit(X_train, Y_train, batch_size=128, epochs=100, validation_data=(X_dev,Y_dev), shuffle=True, callbacks=[tensorboard, checkpoint])